<a href="https://colab.research.google.com/github/adrianb896/PredictiveAnalysis/blob/main/PredictiveAnalysis_Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Predictive Analysis - Assignment 6***
Adrian Bernardino

https://colab.research.google.com/drive/1pon9SlPh2zCscyTbD4uj1Lz1W6KCNpKa?usp=sharing 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
data = pd.read_csv('gdrive/MyDrive/Colab Notebooks/colabPaths/donkeys.csv')
data

,BCS,Age,Sex,Length,Girth,Height,Weight,WeightAlt
0,3.0,<2,stallion,78,90,90,77,NaN
1,2.5,<2,stallion,91,97,94,100,NaN
2,1.5,<2,stallion,74,93,95,74,NaN
3,3.0,<2,female,87,109,96,116,NaN
4,2.5,<2,female,79,98,91,91,NaN
...,...,...,...,...,...,...,...,...
539,3.0,10-15,stallion,98,115,101,145,NaN
540,3.0,10-15,stallion,102,126,110,183,NaN
541,2.5,10-15,stallion,103,118,103,174,NaN
542,3.0,2-5,stallion,91,112,100,139,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   BCS        544 non-null    float64
 1   Age        544 non-null    object 
 2   Sex        544 non-null    object 
 3   Length     544 non-null    int64  
 4   Girth      544 non-null    int64  
 5   Height     544 non-null    int64  
 6   Weight     544 non-null    int64  
 7   WeightAlt  31 non-null     float64
dtypes: float64(2), int64(4), object(2)
memory usage: 34.1+ KB


In [4]:
print("donkeys.csv dataset shape: ", data.shape)

donkeys.csv dataset shape:  (544, 8)


# **Task 1: State the predictive question you would like to answer**

Predictive Question: 
* Can we estimate the weight of a donkey using other featured columns like height to determine the donkeys weight?

# **Identify whether it is a classification or a regression problem and describe the dataset.**

* The problem is dealing with regression since the dataset is having to identify and compare the weights of these donkeys based on the Body Condition Score (BSC), Weight, and WeightAlt. Some quality concerns that will be performed in the next task would be to check the quality of the measurements by comparing the difference between the two weights: *Weight & WeightAlt* and see how consistent the data actually is weighing the donkeys; whether its once or more than once. 

# **Task 2: Data cleaning - Perform at least 1 quality check. Based on your analyses, perform necessary cleaning.**

In [5]:
import plotly.express as px

data['difference (kg)'] = data['WeightAlt'] - data['Weight'] # looking at the difference between both weight features 

data_hist = px.histogram(data, x='difference (kg)', nbins=3, width=550, height=350) # plot 3 bins
# first bin will have a count = 6 with donkeys weighed twice
# second bin will have a count = 21 with donkeys weighed twice
# third bin will have a count = 4 with donkeys weighed twice
data_hist

In [6]:
data['BCS'].value_counts() # distributing the label BCS 

3.0    307
2.5    135
3.5     55
2.0     26
4.0     14
1.5      5
4.5      1
1.0      1
Name: BCS, dtype: int64

From observations we determine that:
* 5 donkeys have a BCS of 1.5 (emaciated)
* 1 donkey has a BCS of 4.5 which is considered roughly (obese)
* 1 donkey has a BCS of 1.0 meaning that they are (emaciated)

In [7]:
exclusive_donkeys = data[(data['BCS'] == 1.0) | (data['BCS'] == 1.5) | (data['BCS'] == 4.5)] # here we are looking for these 7 donkeys specific information by only declaring the BCS of 1.0, 1.5 and 4.5
exclusive_donkeys

,BCS,Age,Sex,Length,Girth,Height,Weight,WeightAlt,difference (kg)
2,1.5,<2,stallion,74,93,95,74,NaN,NaN
5,1.5,<2,female,86,102,98,105,NaN,NaN
163,1.5,5-10,female,92,113,97,131,NaN,NaN
191,1.5,>20,stallion,88,107,95,114,NaN,NaN
212,1.5,15-20,gelding,94,111,105,126,NaN,NaN
291,4.5,10-15,female,107,130,106,227,NaN,NaN
445,1.0,>20,female,97,109,102,115,NaN,NaN


As a result:
* We realize that both donkeys are female

In [8]:
exclusive_donkeys['Weight'].value_counts() # for distribution of the labels

74     1
105    1
131    1
114    1
126    1
227    1
115    1
Name: Weight, dtype: int64

In [9]:
data['WeightAlt'].value_counts() # for distribution of the labels that were NaN

158.0    2
143.0    2
157.0    2
126.0    2
144.0    2
149.0    2
103.0    1
162.0    1
172.0    1
123.0    1
183.0    1
134.0    1
166.0    1
140.0    1
186.0    1
98.0     1
168.0    1
160.0    1
151.0    1
114.0    1
165.0    1
169.0    1
146.0    1
170.0    1
194.0    1
Name: WeightAlt, dtype: int64

# **Task 3: Feature Engineering - Prepare the data for machine learning algorithms by performing necessary transformations on the data**

In [10]:
def remove_1(data):
    return data[(data['BCS'] >= 1.5) & (data['BCS'] <= 4)] # tranforming our data by removing bcs outliers

data = data.pipe(remove_1) # performing .pipe() method to remove these 2 

In [11]:
donkey_hist = px.histogram(data, x='Weight', nbins=20, width=550, height=350) # plotting the weight of the donkeys
donkey_hist

Now to identify the relationship between the height and weight by plotting a scatter plot.

In [12]:
donkey_scatter = px.scatter(data, x='Height', y='Weight', width=550, height=350)
donkey_scatter

In [13]:
# as shown, we notice that 1 lightweight donkey does not weigh close to the average.
# we also notice that 1 heavyweight donkey is also a bit separated from the average. 
def remove_2(data):
  return data[(data['Weight'] >= 40)] # tranforming our data by removing weight outliers

data = (data.pipe(remove_1).pipe(remove_2)) # updating our data set by adding both functions

In [14]:
data.shape

(541, 9)

# **Task 4:**
* **Perform train-test-split**
* **Training - Train a machine learning model on the transformed data**
* **Evaluation - Evaluate the model on the test set**

In [15]:
# performing a randomization for the indices
import numpy as np

np.random.seed(50)
num = len(data)
random_indices = np.arange(num)
np.random.shuffle(random_indices)
num_train = int(np.round((0.7 * num)))

In [16]:
x_train = data.iloc[random_indices[:num_train]] # 70
x_test = data.iloc[random_indices[num_train:]] # 30

70/30 split

In [17]:
x_train.shape # 70% for training set

(379, 9)

In [18]:
x_test.shape # 30% for testing set

(162, 9)

By oberservation we can see that girth column has the best linear association with the weight column in terms of its relationship compared to the other columns. 

In [19]:
train = x_train[['Weight', 'Length', 'Girth', 'Height']] # quantitative variables comparing with Weight
train.corr() # finding the pairwise correlation

,Weight,Length,Girth,Height
Weight,1.000000,0.793852,0.907038,0.727064
Length,0.793852,1.000000,0.694904,0.589163
Girth,0.907038,0.694904,1.000000,0.714789
Height,0.727064,0.589163,0.714789,1.000000


By modeling a donkeys weight, we need to determine a loss function for when the vet needs to figure our how much the donkey weighs in order to prescribe the right amount of medicine. In this case anesthetics. This prevents the vets from overdosing the donkeys which leads to a successful and individual prescriptions for each donkey.  

Relative error: *100(y - y_hat)/y_hat* 
* y = true-positive value
* y_hat = prediction value

In [20]:
import matplotlib.pyplot as plt

def loss_function(x):
  y = (x >= 0) + 3 * (x < 0)
  return np.square(x) * y

x_ticks = np.linspace(-40, 40, 500)
loss_anes = loss_function(x_ticks)

fig = px.line(x=x_ticks, y=loss_anes, title='Quadratic Loss Function', width=550, height=350)
fig.show()
# y representing the loss
# x representing the relative error percentage


A linear model can be established as well to determine girth which had the most correlation with weight column. 
$$
f_{\theta}(x) = \theta_0 + \theta_1 \cdot \text{Girth}
$$


In [21]:
# establishing a matrix for girth
x = x_train.assign(intr=1)[['intr', 'Girth']]
x

,intr,Girth
122,1,116
282,1,125
361,1,111
116,1,112
415,1,121
...,...,...
143,1,116
495,1,123
119,1,129
14,1,110


In [22]:
# establishing a matrix for weight
y = x_train['Weight']
y

122    154
282    199
361    126
116    142
415    178
      ... 
143    167
495    177
119    163
14     112
166    165
Name: Weight, Length: 379, dtype: int64

In [23]:
from scipy.optimize import minimize

def training_loss_anes(x, y): # declaring minimization for loss_function
    def loss_theta(theta):
        predicted_value = x @ theta
        return np.mean(loss_function(100 * (y - predicted_value) / predicted_value)) # approaching an algorithm based in linear models
    return loss_theta

output = minimize(training_loss_anes(x, y), np.ones(2))
theta_hat = output['x']

print('After fitting:')
print(f'theta_0 = {theta_hat[0]:>7.2f}')
print(f'theta_1 = {theta_hat[1]:>7.2f}')

After fitting:
theta_0 = -227.26
theta_1 =    3.24


In [24]:
predicted_value = x @ theta_hat
residual = 100 * (y - predicted_value) / predicted_value # predicting the weights for each donkeys as error percentage

residual_plot = pd.DataFrame({'Predicted Weight': predicted_value, 'Error (%)': residual}) # scatter plot displaying residual for our training set
px.scatter(residual_plot, x='Predicted Weight', y='Error (%)', width=550, height=350)

Multiple linear model needs to be applied in order to establish relationship between girth, length, and height and improve our predictions

In [25]:
import itertools

num = ['Girth', 'Length', 'Height'] # showing the total combinations by using itertools to print out a list
combos = [list(subset)
          for i in [1, 2, 3] # 3 combinations
          for subset in itertools.combinations(num, i)] # go through each num and combine as many ways possible wihtout repeating
combos

[['Girth'],
 ['Length'],
 ['Height'],
 ['Girth', 'Length'],
 ['Girth', 'Height'],
 ['Length', 'Height'],
 ['Girth', 'Length', 'Height']]

In [26]:

def mean_training_error(model_comb):
    x = x_train.assign(intr=1)[['intr', *model_comb]]
    theta_hat = minimize(training_loss_anes(x, y), np.ones(x.shape[1]))['x']
    predicted_value = x @ theta_hat
    return np.mean(loss_function(100 * (y - predicted_value)/ predicted_value))

model_risks = [
    mean_training_error(model_comb)
    for model_comb in combos
]


Tranforming the training set into dummy variables by performing one-hot-encoding imputation

In [27]:
ohe = (x_train.assign(intr=1)[['intr', 'Length', 'Girth', 'BCS', 'Age', 'Sex']]
    .pipe(pd.get_dummies, columns=['BCS', 'Age', 'Sex'])
    .drop(columns=['BCS_3.0', 'Age_5-10', 'Sex_female']))
ohe

,intr,Length,Girth,BCS_1.5,BCS_2.0,BCS_2.5,BCS_3.5,BCS_4.0,Age_10-15,Age_15-20,Age_2-5,Age_<2,Age_>20,Sex_gelding,Sex_stallion
122,1,97,116,0,0,0,0,0,1,0,0,0,0,1,0
282,1,105,125,0,0,0,0,1,1,0,0,0,0,0,1
361,1,81,111,0,0,0,0,0,0,0,0,0,0,0,1
116,1,97,112,0,0,0,0,0,0,0,0,0,0,0,0
415,1,101,121,0,0,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,1,97,116,0,0,0,0,0,0,0,0,0,0,0,0
495,1,108,123,0,0,0,0,0,0,1,0,0,0,0,1
119,1,89,129,0,0,0,0,0,0,0,0,0,1,0,0
14,1,87,110,0,0,1,0,0,0,0,0,1,0,0,0


In [28]:
# categorical features included to fit the model 
output = minimize(training_loss_anes(ohe, y), np.ones(ohe.shape[1]))

theta_hat = output['x']
y_prediction = ohe @ theta_hat # ohe values iterated

training_error = (np.mean(loss_function(100 * (y - y_prediction)/ y_prediction))) 
print(f'Training Error: {training_error:.2f}')

Training Error: 50.91


Observation: 
* Multi linear model did better than our simple linear model

In [29]:
# applying the dummy variables from ohe to make the model simpler
bcs = ['BCS_1.5', 'BCS_2.0', 'BCS_2.5', 'BCS_3.5', 'BCS_4.0']
age = ['Age_<2', 'Age_2-5', 'Age_10-15', 'Age_15-20', 'Age_>20']
sex = ['Sex_gelding', 'Sex_stallion']

thetas_vars = (pd.DataFrame({'var': ohe.columns, 'theta_hat': theta_hat})
          .set_index('var'))

bcs_scatter = px.scatter(thetas_vars.loc[bcs].reset_index(), x='var', y='theta_hat', width=550, height=350)
sex_scatter = px.scatter(thetas_vars.loc[sex].reset_index(), x='var', y='theta_hat', width=550, height=350)
age_scatter = px.scatter(thetas_vars.loc[age].reset_index(), x='var', y='theta_hat', width=550, height=350)


In [30]:
bcs_scatter.show()

In [31]:
sex_scatter.show()

In [32]:
age_scatter.show()

In [33]:
# updating the matrix to refit the simpler model
def bcs_merge(x):
    new_bcs = x['BCS_2.0'] + x['BCS_1.5']
    return x.assign(**{'BCS_2.0': new_bcs}).drop(columns=['BCS_1.5'])

def age_sex(x):
    return x.drop(columns=['Age_10-15', 'Age_15-20', 'Age_>20', 'Sex_gelding', 'Sex_stallion'])

ohe_simple_model = (ohe.pipe(bcs_merge).pipe(age_sex))

In [34]:
outputs = minimize(training_loss_anes(ohe_simple_model, y), np.ones(ohe_simple_model.shape[1]))

theta_hat = outputs['x']
y_pred = ohe_simple_model @ theta_hat

training_error = (np.mean(loss_function(100 * (y - y_pred)/ y_pred)))
print(f'Training error: {training_error:.2f}')

Training error: 52.23


the average training error is more complex than the previous one we created. 

In [35]:
thetas = pd.DataFrame({'var': ohe_simple_model.columns, 'theta_hat': theta_hat})
thetas.head(10)

,var,theta_hat
0,intr,-175.355235
1,Length,0.903010
2,Girth,2.063905
3,BCS_2.0,-4.912581
4,BCS_2.5,-5.119413
5,BCS_3.5,8.193148
6,BCS_4.0,15.229972
7,Age_2-5,-4.007848
8,Age_<2,-7.379981


Modeling our assesment was successfule for the training set which allowed us to put aside 70%. However by splitting the dataset into 70/30 split, we were able to put aside 30% of our data.

In our test set we will be applying what was done in the training set and use it in the testing set to predict the weights of the donkeys. An appropiate method is using our dummy variables to combine with our girth and length columns of the donkey dataset.

In [36]:
y_test = data['Weight']

x_test = (data.assign(intr=1)[['intr', 'Length', 'Girth', 'BCS', 'Age', 'Sex']] # applying our data to our testing set
          .pipe(pd.get_dummies, columns=['BCS', 'Age', 'Sex']) # use the dummy variables for the donkeys age, sex, and body condition score
          .drop(columns=['BCS_3.0', 'Age_5-10', 'Sex_female']) # drop columns that need consolidating
          .pipe(bcs_merge) # remove specifics
          .pipe(age_sex)) # remove specifics

In [37]:
# implementing the training sets theta variables to make weight predictions
y_test_p = x_test @ theta_hat 
test_error = 100 * (y_test - y_test_p) / y_test_p # loss function
print(f'Validation error: \n')
test_error

Validation error: 



0       4.831786
1       5.799889
2       4.052279
3      -3.967377
4       6.127600
         ...    
539    -3.647320
540     3.504563
541    11.484035
542     3.755987
543     8.320622
Length: 541, dtype: float64

In [38]:
fig = px.scatter(x=y_test_p, y=test_error, title='Test Set Predictions', width=550, height=350)
fig.add_hline(10, line_width=3, line_color='orange')
fig.add_hline(-10, line_width=3, line_color='orange')
fig.show()

In [39]:
import plotly.graph_objects as go

# declaring our linear model
fig = px.scatter(y=y_test, x=y_test_p, title="Test Set Predictions", width=750, height=350)

fig.add_trace(go.Scatter(x=[60, 200], y=[60, 200], name='0% error', mode='lines', marker=dict(color='yellow'))) # this shows the average estimation
fig.add_trace(go.Scatter(x=[60, 200], y=[66, 220], name='10% error', mode='lines', marker=dict(color='orange'))) # this shows the underestimation
fig.add_trace(go.Scatter(x=[60, 200], y=[54, 180], name='-10% error', mode='lines', marker=dict(color='red'))) # this shows the overestimation
fig.show()
# x axis representing the predicted weight
# y axis representing the actual weight

# **Task 5: Present any conclusion from the results**

# In conclusion, the results I found in my predictive analysis was that performing train-test-split is key to being able to build models that can be intepreted easily. This gave me a good idea as to how I need to distribute and set aside data to configure the models being built based on the transformation data. As we can see, we are able to implement different features to find the prediction for the weights. Here in my models I wanted to apply a linear model that allows me to fit training data. This can lead to having a high bias and low training error which falls in an underfitting model. I used the linear approach in my testing set because of the generalization that occurs when looking at our test errors. I feel like with my results im able to have this data work well when trying to implement new data. Having it work well also leads to low variance. 